In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import numpy as np
import os
import time

     |████████████████████████████████| 332.1MB 65kB/s 
     |████████████████████████████████| 3.0MB 39.8MB/s 
     |████████████████████████████████| 61kB 27.6MB/s 
     |████████████████████████████████| 419kB 58.6MB/s 


In [0]:
# Upload Migos Text File
from google.colab import files
uploaded = files.upload()

Saving migos-bad-and-bougie.txt to migos-bad-and-bougie.txt


In [0]:
# This RNN uses individual characters to enagage with text - how many characters are there?
text = uploaded["migos-bad-and-bougie.txt"].decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 901843 characters


In [0]:
# Sample Text
print(text[:250])

[Offset:]
You know young rich niggas
You know so we never really had no old money
We got a whole lot of new money though, hah
(If Young Metro don't trust you I'm gon' shoot ya)
Hey

Raindrop, drop, drop top (drop top)
Smokin' on cookie in the hotbox 


In [0]:
# Creating a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
# View the character mapping
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [0]:
# Example of a mapping of Migos lyrics to a coded array
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:25]), text_as_int[:25]))

'[Offset:]\nYou know young ' ---- characters mapped to int ---- > [56 44 64 64 77 63 78 26 57  0 54 73 79  1 69 72 73 81  1 83 73 79 72 65
  1]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(7):
  print(idx2char[i.numpy()])

[
O
f
f
s
e
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'[Offset:]\nYou know young rich niggas\nYou know so we never really had no old money\nWe got a whole lot '
"of new money though, hah\n(If Young Metro don't trust you I'm gon' shoot ya)\nHey\n\nRaindrop, drop, drop"
" top (drop top)\nSmokin' on cookie in the hotbox (cookie)\nFuckin' on your bitch she a thot, thot, thot"
"\nCookin' up dope in the crockpot (pot)\nWe came from nothin' to somethin' nigga (hey)\nI don't trust no"
'body grip the trigger (nobody)\nCall up the gang, and they come and get you (gang)\nCry me a river, giv'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[Offset:]\nYou know young rich niggas\nYou know so we never really had no old money\nWe got a whole lot'
Target data: 'Offset:]\nYou know young rich niggas\nYou know so we never really had no old money\nWe got a whole lot '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 56 ('[')
  expected output: 44 ('O')
Step    1
  input: 44 ('O')
  expected output: 64 ('f')
Step    2
  input: 64 ('f')
  expected output: 64 ('f')
Step    3
  input: 64 ('f')
  expected output: 77 ('s')
Step    4
  input: 77 ('s')
  expected output: 63 ('e')


In [0]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
# Neural Network Architecture
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
     tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    #tf.keras.layers.LSTM(rnn_units,
     #                   return_sequences=True,
      #                  stateful=True,
       #                 recurrent_initializer='glorot_uniform'),  
    tf.keras.layers.Dense(vocab_size),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

W0531 17:27:32.396944 139664719816576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f061d6fc9e8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0531 17:27:32.409647 139664719816576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f05cc52ba20>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
for input_example_batch, target_example_batch in dataset.take(2):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 103) # (batch_size, sequence_length, vocab_size)
(32, 100, 103) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           26368     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (32, None, 1024)          5246976   
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (32, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (32, None, 103)           105575    
_________________________________________________________________
dense_1 (Dense)              (32, None, 103)           10712     
Total params: 13,782,335
Trainable params: 13,782,335
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([102,  23,   6,  45,  60,  86,  43,   0,  25,  66,  28,   0,  96,
        10, 100,  20,  61,  42,  42,  72,  49,   1,  46,  45,  21,  89,
        80,  38,  53,  91,  39,  82,   1,  47,  55,  81,  30,  73,  22,
        75,  77,  25,  90,  28,   1,  16, 101,   8,  84,   8,  15,  75,
        41,  89,  20,  11,   2,  74,  31,   5,  68,  82,  48,  24,  91,
        91,  84,  78,  54,  70,  40,  54,   9,  33,  54,  61,  51,  99,
        83,  47,  91,  19,  10, 101,  41,  24,  80,  41,  90,  56,  98,
        71,  24,  20,  93,  23,  25,  70,  17,  96])

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 103)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.6353507


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
EPOCHS = 75
#EPOCHS=100

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
# Train Model - Takes about Half an Hour on a GPU
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/75
279/279 [==============================] - 29s 104ms/step - loss: 2.6880
Epoch 2/75
279/279 [==============================] - 29s 105ms/step - loss: 1.7235
Epoch 3/75
279/279 [==============================] - 30s 109ms/step - loss: 1.4487
Epoch 4/75
279/279 [==============================] - 31s 112ms/step - loss: 1.2905
Epoch 5/75
279/279 [==============================] - 30s 107ms/step - loss: 1.1605
Epoch 6/75
279/279 [==============================] - 31s 110ms/step - loss: 1.0415
Epoch 7/75
279/279 [==============================] - 31s 110ms/step - loss: 0.9372
Epoch 8/75
279/279 [==============================] - 31s 110ms/step - loss: 0.8431
Epoch 9/75
279/279 [==============================] - 31s 110ms/step - loss: 0.7543
Epoch 10/75
279/279 [==============================] - 31s 111ms/step - loss: 0.6673
Epoch 11/75
279/279 [==============================] - 31s 110ms/step - loss: 0.5782
Epoch 12/75
279/279 [==============================] - 31s 111ms/step - lo

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_75'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

W0531 18:07:06.230292 139664719816576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f059a5e6fd0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0531 18:07:06.235155 139664719816576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f059a5f8ac8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            26368     
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (1, None, 1024)           5246976   
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (1, None, 1024)           8392704   
_________________________________________________________________
dense_2 (Dense)              (1, None, 103)            105575    
_________________________________________________________________
dense_3 (Dense)              (1, None, 103)            10712     
Total params: 13,782,335
Trainable params: 13,782,335
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.1

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
# Example Song!
print(generate_text(model, start_string=u"[Hook] \n"))

[Hook] 
[Verse 3: Offset]
You mught me talking Brokanese, you split em by the bando, know the with them bricks from them boys count I come in my trap
But it's all good though

[Hook]

[Verse 2 - Takeoff:]
Takeoff!!
When I beat the money I poured her (Trap, andn's your ass up right witcha)
Get right witcha (I'm a get right witcha)
Bad bitches, fuck 'em then dismiss em (bad, woo)
I ain't really here to take no pictures (flash)
Middle finger up fuck the system (fuck 'em)

Yeah, coupe is ride, yeah
Pull up on mistoli, invented that your brother
You say you on the block skuuuurt)
I'm not gonna tell to then you in Range
Now when I wake up I lost my brother T
Welr man, who who eart
Oh, what is the perkinjachip
Straight up out the Camble
Wrist spinning like a motor
IPhout out the seal, we pour out the seal
I ran out of lean, I need a refill
We pour out the seal, we pour out the seal
I ran out of lean, I need a refill
We pour out the seal, we pour out the seal
I don't wanna feel down Rollie she